# Business Metrics - Excel to SQLite 변환 및 서버 업로드

구글 드라이브의 엑셀 파일을 SQLite로 변환하고 서버에 업로드

In [ ]:
!pip install -q openpyxl requests

In [ ]:
import sqlite3
import os
import json
from pathlib import Path
from openpyxl import load_workbook
import requests
from google.colab import drive
import time

In [ ]:
# ============ 설정 ============
SERVER_CONFIG = {
    'main': {'url': 'http://14.7.14.31:6001/api/upload-db', 'name': '본 사이트'},
    'demo': {'url': 'http://14.7.14.31:6005/api/upload-db', 'name': '데모 사이트'}
}

USE_SERVER = 'demo'  # 'main' 또는 'demo'
API_KEY = 'biofl1411-upload-key'

# 드라이브 경로 (실제 구조에 맞게 설정)
DRIVE_BASE_PATH = '/content/drive/MyDrive/ilabdata'

# 변환할 연도
YEARS = ['2023', '2024', '2025', '2026']

# 출력 DB 파일
OUTPUT_DB = '/content/business_data.db'

print(f"서버: {SERVER_CONFIG[USE_SERVER]['name']}")
print(f"드라이브: {DRIVE_BASE_PATH}")
print(f"연도: {YEARS}")

In [ ]:
# 드라이브 마운트
drive.mount('/content/drive')

# 폴더 확인
base = Path(DRIVE_BASE_PATH)
print(f"\n=== 폴더 확인 ===")
for folder in ['food_all', 'food_item']:
    folder_path = base / folder
    if folder_path.exists():
        subfolders = list(folder_path.iterdir())
        print(f"{folder}: {[f.name for f in subfolders]}")
    else:
        print(f"{folder}: 없음")

In [ ]:
def init_sqlite_db(db_path):
    """SQLite DB 초기화"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS excel_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            year TEXT, 접수번호 TEXT, 접수일자 TEXT, 발행일 TEXT, 검체유형 TEXT,
            업체명 TEXT, 의뢰인명 TEXT, 업체주소 TEXT, 영업담당 TEXT, 검사목적 TEXT,
            총금액 REAL, 시험분야 TEXT, 입금일 TEXT, 입금여부 TEXT, 검사구분 TEXT,
            입금구분 TEXT, 업체분류 TEXT, raw_data TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS food_item_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            year TEXT, 접수일자 TEXT, 발행일 TEXT, 검체유형 TEXT, 업체명 TEXT,
            의뢰인명 TEXT, 업체주소 TEXT, 항목명 TEXT, 규격 TEXT, 항목담당 TEXT,
            결과입력자 TEXT, 입력일 TEXT, 분석일 TEXT, 항목단위 TEXT, 시험결과 TEXT,
            시험치 TEXT, 성적서결과 TEXT, 판정 TEXT, 검사목적 TEXT, 긴급여부 TEXT,
            항목수수료 REAL, 영업담당 TEXT
        )
    ''')

    cursor.execute('CREATE TABLE IF NOT EXISTS file_metadata (file_path TEXT PRIMARY KEY, mtime REAL, row_count INTEGER)')
    cursor.execute('CREATE TABLE IF NOT EXISTS token_usage (id INTEGER PRIMARY KEY, date TEXT, year_month TEXT, model TEXT, input_tokens INTEGER, output_tokens INTEGER, total_tokens INTEGER, cost_usd REAL, cost_krw REAL, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)')

    cursor.execute('CREATE INDEX IF NOT EXISTS idx_excel_year ON excel_data(year)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_food_year ON food_item_data(year)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_food_item ON food_item_data(항목명)')

    conn.commit()
    conn.close()
    print(f"DB 초기화 완료: {db_path}")

In [ ]:
def convert_food_all(conn, base_path, years):
    """food_all 데이터 변환 (excel_data 테이블)"""
    cursor = conn.cursor()
    total = 0

    for year in years:
        # 폴더명: food_all_2024
        data_path = Path(base_path) / 'food_all' / f'food_all_{year}'
        if not data_path.exists():
            print(f"food_all_{year}: 폴더 없음")
            continue

        files = sorted(data_path.glob("*.xlsx"))
        if not files:
            print(f"food_all_{year}: 파일 없음")
            continue

        print(f"\nfood_all_{year}: {len(files)}개 파일")
        cursor.execute('DELETE FROM excel_data WHERE year = ?', (year,))

        for f in files:
            try:
                wb = load_workbook(f, read_only=True, data_only=True)
                ws = wb.active
                headers = [cell.value for cell in ws[1]]

                batch = []
                for row in ws.iter_rows(min_row=2, values_only=True):
                    d = dict(zip(headers, row))
                    batch.append((
                        year,
                        str(d.get('접수번호', '') or ''),
                        str(d.get('접수일자', '') or ''),
                        str(d.get('발행일', '') or ''),
                        str(d.get('검체유형', '') or ''),
                        str(d.get('업체명', '') or ''),
                        str(d.get('의뢰인명', '') or ''),
                        str(d.get('업체주소', '') or ''),
                        str(d.get('영업담당', '') or ''),
                        str(d.get('검사목적', '') or ''),
                        float(d.get('총금액', 0) or 0),
                        str(d.get('시험분야', '') or ''),
                        str(d.get('입금일', '') or ''),
                        str(d.get('입금여부', '') or ''),
                        str(d.get('검사구분', '') or ''),
                        str(d.get('입금구분', '') or ''),
                        str(d.get('업체분류', '') or ''),
                        json.dumps(d, ensure_ascii=False, default=str)
                    ))

                cursor.executemany('''
                    INSERT INTO excel_data (year, 접수번호, 접수일자, 발행일, 검체유형, 업체명, 의뢰인명, 업체주소, 영업담당, 검사목적, 총금액, 시험분야, 입금일, 입금여부, 검사구분, 입금구분, 업체분류, raw_data)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', batch)
                wb.close()
                total += len(batch)
                print(f"  {f.name}: {len(batch):,}건")
            except Exception as e:
                print(f"  {f.name}: 오류 - {e}")

    conn.commit()
    return total

In [ ]:
def convert_food_item(conn, base_path, years):
    """food_item 데이터 변환"""
    cursor = conn.cursor()
    total = 0
    cols = ['접수일자', '발행일', '검체유형', '업체명', '의뢰인명', '업체주소', '항목명', '규격', '항목담당', '결과입력자', '입력일', '분석일', '항목단위', '시험결과', '시험치', '성적서결과', '판정', '검사목적', '긴급여부', '항목수수료', '영업담당']

    for year in years:
        # 폴더명: food_item_2024
        data_path = Path(base_path) / 'food_item' / f'food_item_{year}'
        if not data_path.exists():
            print(f"food_item_{year}: 폴더 없음")
            continue

        files = sorted(data_path.glob("*.xlsx"))
        if not files:
            print(f"food_item_{year}: 파일 없음")
            continue

        print(f"\nfood_item_{year}: {len(files)}개 파일")
        cursor.execute('DELETE FROM food_item_data WHERE year = ?', (year,))

        for f in files:
            try:
                wb = load_workbook(f, read_only=True, data_only=True)
                ws = wb.active
                headers = [cell.value for cell in ws[1]]
                col_idx = {h: i for i, h in enumerate(headers) if h in cols}

                batch = []
                for row in ws.iter_rows(min_row=2, values_only=True):
                    d = {c: row[i] if i < len(row) else None for c, i in col_idx.items()}
                    batch.append((
                        year,
                        str(d.get('접수일자', '') or ''),
                        str(d.get('발행일', '') or ''),
                        str(d.get('검체유형', '') or ''),
                        str(d.get('업체명', '') or ''),
                        str(d.get('의뢰인명', '') or ''),
                        str(d.get('업체주소', '') or ''),
                        str(d.get('항목명', '') or ''),
                        str(d.get('규격', '') or ''),
                        str(d.get('항목담당', '') or ''),
                        str(d.get('결과입력자', '') or ''),
                        str(d.get('입력일', '') or ''),
                        str(d.get('분석일', '') or ''),
                        str(d.get('항목단위', '') or ''),
                        str(d.get('시험결과', '') or ''),
                        str(d.get('시험치', '') or ''),
                        str(d.get('성적서결과', '') or ''),
                        str(d.get('판정', '') or ''),
                        str(d.get('검사목적', '') or ''),
                        str(d.get('긴급여부', '') or ''),
                        float(d.get('항목수수료', 0) or 0),
                        str(d.get('영업담당', '') or '')
                    ))

                cursor.executemany('''
                    INSERT INTO food_item_data (year, 접수일자, 발행일, 검체유형, 업체명, 의뢰인명, 업체주소, 항목명, 규격, 항목담당, 결과입력자, 입력일, 분석일, 항목단위, 시험결과, 시험치, 성적서결과, 판정, 검사목적, 긴급여부, 항목수수료, 영업담당)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', batch)
                wb.close()
                total += len(batch)
                print(f"  {f.name}: {len(batch):,}건")
            except Exception as e:
                print(f"  {f.name}: 오류 - {e}")

    conn.commit()
    return total

In [ ]:
# 변환 실행
if os.path.exists(OUTPUT_DB):
    os.remove(OUTPUT_DB)

init_sqlite_db(OUTPUT_DB)
conn = sqlite3.connect(OUTPUT_DB)

print("="*50)
print("변환 시작")
print("="*50)

start = time.time()
excel_cnt = convert_food_all(conn, DRIVE_BASE_PATH, YEARS)
food_cnt = convert_food_item(conn, DRIVE_BASE_PATH, YEARS)
conn.close()

print("\n" + "="*50)
print(f"완료! excel_data: {excel_cnt:,}건, food_item_data: {food_cnt:,}건")
print(f"시간: {time.time()-start:.1f}초, 크기: {os.path.getsize(OUTPUT_DB)/1024/1024:.1f}MB")

In [ ]:
# DB 검증
conn = sqlite3.connect(OUTPUT_DB)
cursor = conn.cursor()

for table in ['excel_data', 'food_item_data']:
    cursor.execute(f"SELECT year, COUNT(*) FROM {table} GROUP BY year ORDER BY year")
    print(f"\n{table}:")
    for year, cnt in cursor.fetchall():
        print(f"  {year}년: {cnt:,}건")

conn.close()

In [ ]:
def upload_to_server(db_path, server_key='demo'):
    """서버 업로드"""
    url = SERVER_CONFIG[server_key]['url']
    name = SERVER_CONFIG[server_key]['name']
    print(f"\n{name}에 업로드 중... ({url})")

    try:
        with open(db_path, 'rb') as f:
            resp = requests.post(url, files={'file': ('business_data.db', f)}, headers={'X-API-Key': API_KEY}, timeout=300)
        if resp.status_code == 200:
            result = resp.json()
            print(f"업로드 성공!")
            for t, c in result.get('tables', {}).items():
                print(f"  {t}: {c:,}건")
            return True
        else:
            print(f"실패 ({resp.status_code}): {resp.text}")
    except Exception as e:
        print(f"오류: {e}")
    return False

In [ ]:
# 데모 서버 업로드
upload_to_server(OUTPUT_DB, 'demo')

In [ ]:
# 본 사이트 업로드 (데모 테스트 후)
# upload_to_server(OUTPUT_DB, 'main')